In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
batch_size, img_height, img_width = 64, 180, 180

training_dir = tf.keras.preprocessing.image_dataset_from_directory(
    directory = data_dir,
    validation_split = 0.2,
    subset = "training",
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size)

validation_dir = tf.keras.preprocessing.image_dataset_from_directory(
    directory = data_dir,
    validation_split = 0.2,
    subset = "validation",
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size)

In [ ]:
def get_training_dir(directory, batch_size, img_height, img_width, validation_split, seed):
    training_dir = tf.keras.preprocessing.image_dataset_from_directory(
        directory = directory,
        validation_split = validation_split,
        image_size = (img_height, img_width),
        subset = "training",
        seed = seed,
        batch_size = batch_size)
    return training_dir

def get_validation_dir(directory, batch_size, img_height, img_width, validation_split, seed):
    validation_dir = tf.keras.preprocessing.image_dataset_from_directory(
        directory = directory,
        validation_split = validation_split,
        image_size = (img_height, img_width),
        batch_size = batch_size,
        seed = seed,
        subset = "validation")
    return validation_dir 

training_dir = get_training_dir(data_dir, 64, 180, 180, 0.2, 123)
validation_dir = get_validation_dir(data_dir, 64, 180, 180, 0.2, 123)

In [ ]:
class_names = training_dir.class_names
print("There are:", len(class_names), "classes \nand their names are: ", class_names)

for images, labels in training_dir:
    print(images.shape)
    print(labels.shape)
    break

In [ ]:
#prefetch overlaps data preprocessing and model execution while training.
#cache keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.

AUTOTUNE = tf.data.experimental.AUTOTUNE
training_dir = training_dir.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
validation_dir = validation_dir.cache().prefetch(buffer_size = AUTOTUNE)

normalizing_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = training_dir.map(lambda x, y: (normalizing_layer(x), y))

image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
num_classes = len(class_names)

data_aug = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
    input_shape = (img_height, img_width)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)])

model = tf.keras.Sequential([
    data_aug,
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.Conv2D(64, (3, 3), activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2, 2), 
    tf.keras.layers.Conv2D(64, (3, 3), activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2, 2), 

    tf.keras.layers.Conv2D(64, (3, 3), activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2, 2), 
    tf.keras.layers.Conv2D(64, (3, 3), activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = tf.nn.relu),
    tf.keras.layers.Dense(num_classes, activation = tf.nn.softmax)
])

model = tf.keras.Input()

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
training_dir = training_dir.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
validation_dir = validation_dir.cache().prefetch(buffer_size = AUTOTUNE)

In [ ]:
model.summary()

In [ ]:
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])

In [ ]:
epochs = 10
history = model.fit(training_dir, validation_data = validation_dir, epochs = epochs)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize = (8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label = "Training Accuracy")
plt.plot(epochs_range, val_acc, label = "Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = "Training Loss")
plt.plot(epochs_range, val_loss, label = "Validation Loss")